# 4. Running style transfer at scale
This notebook will take us through what it looks like to run neural style transfer at scale in Azure using Batch AI. 


---

Import utilities. For this notebook, we're going to use some of the utilities provided as part of this package.

In [ ]:
import os
from az.util import bai
%load_ext dotenv
%dotenv

## Setting up your cluster
This section will walk through setting up your cluster and some of the parameters you can use to customize your cluster.

Using the provided utility, set up the Batch AI client:

In [ ]:
client = bai.setup_bai()

Before creating the cluster, we need to set up parameters for our cluster:

In [ ]:
workspace = "<my-workspace>"
cluster_name = "<my-cluster-name>"
cluster_vm_size = "<vm-size>"
cluster_vm_priority = "<vm-priority>"
cluster_minimum_node_count = "<minimum-node-count>"
cluster_maximum_node_count = "<maximum-node-count>"
cluster_initial_node_count = "<initial-node-count>"
cluster_container_mnt_path = "<container-mnt-path>"
admin_user_name = "<username>"
admin_user_password = "<my-secret-password>"

Save the cluster settings to the .env file.

In [ ]:
!dotenv set WORKSPACE $workspace
!dotenv set CLUSTER_NAME $cluster_name
!dotenv set CLUSTER_VM_SIZE $cluster_vm_size
!dotenv set CLUSTER_VM_PRIORITY $cluster_vm_priority
!dotenv set CLUSTER_MINIMUM_NODE_COUNT $cluster_minimum_node_count
!dotenv set CLUSTER_MAXIMUM_NODE_COUNT $cluster_maximum_node_count
!dotenv set CLUSTER_INITIAL_NODE_COUNT $cluster_initial_node_count
!dotenv set CLUSTER_CONTAINER_MNT_PATH $cluster_container_mnt_path
!dotenv set ADMIN_USER_NAME $admin_user_name
!dotenv set ADMIN_USER_PASSWORD $admin_user_password

In [ ]:
%reload_ext dotenv
%dotenv

Using the Batch AI client, we can set up the cluster. In Batch AI, clusters must belong within a _workspace_. Next, create both the workspace and cluster: 

In [ ]:
bai.create_workspace(client, ws=workspace)
bai.create_autoscale_cluster(client, cluster_name)

Take a look at the state of the cluster.

NOTE: This can also be done by going into the portal and inspecting the cluster via the Batch AI UI.

In [ ]:
cluster = bai.get_cluster(client, cluster_name, ws=workspace)
print(('Cluster state: {0}; Allocated: {1}; Idle: {2}; ' +
     'Unusable: {3}; Running: {4}; Preparing: {5}; ' +
     'Leaving: {6}').format(
        cluster.allocation_state,
        cluster.current_node_count,
        cluster.node_state_counts.idle_node_count,
        cluster.node_state_counts.unusable_node_count,
        cluster.node_state_counts.running_node_count,
        cluster.node_state_counts.preparing_node_count,
        cluster.node_state_counts.leaving_node_count))

## Running a job on the cluster
This section of the notebook will walk through how to run a Batch AI job on the cluster we just created. This section will first use AzCopy to upload our individual frames from the video onto blob storage. It will also use AzCopy to copy over the style transfer script and the style image.

After that, we will primarily be relying on the script __create_job.py__ to create jobs on Batch AI.

First we use AzCopy to upload the following items into the storage account we've created:

- The style image - 
- The style transfer script
- The directory with all the individual frames 

In [ ]:
fs_script = "script.py"
fs_style_image = "style_image.py"
fs_content_dir = "orangutan"

In [ ]:
!dotenv set FS_SCRIPT $fs_script
!dotenv set FS_STYLE_IMAGE $fs_style_image
!dotenv set FS_CONTENT_DIR $fs_content_dir

In [ ]:
%reload_ext dotenv
%dotenv

In [ ]:
%%bash
azcopy \
    --source pytorch/style_transfer_script.py \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_SCRIPT} \
    --dest-key $STORAGE_ACCOUNT_KEY
    
azcopy \
    --source pytorch/images/style_images/sample_renior.jpg \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_STYLE_IMAGE} \
    --dest-key $STORAGE_ACCOUNT_KEY
    
azcopy \
    --source pytorch/images/${VIDEO_NAME} \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_CONTENT_DIR} \
    --dest-key $STORAGE_ACCOUNT_KEY \
    --recursive

In [ ]:
experiment_prefix = "exp"
job_name_prefix = "job"
job_node_count = 1
job_batch_size = 50
fs_output_dir_prefix = "output"
fs_logger_dir_prefix = "log"

In [ ]:
!dotenv set EXPERIMENT_PREFIX $experiment_prefix
!dotenv set JOB_NAME_PREFIX $job_name_prefix
!dotenv set JOB_NODE_COUNT $job_node_count
!dotenv set JOB_BATCH_SIZE $job_batch_size
!dotenv set FS_OUTPUT_DIR_PREFIX $fs_output_dir_prefix
!dotenv set FS_LOGGER_DIR_PREFIX $fs_logger_dir_prefix

In [ ]:
%reload_ext dotenv
%dotenv

In [ ]:
!python az/create_job.py --job-batch-size 50 --content-images-blob-dir $fs_content_dir

When the jobs finish running, you can use the Azure portal or Storage explorer to inspect the output images.

Inside your Blob Container, you should notice that a new directory with the datetime-stamp is created. Output images are stored there.

---

## Conclusion
[todo]